<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat Completion for Decisions 

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.33.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.0 Microsoft.SemanticKernel, 1.33.0

Using Azure OpenAI Service


### Step 2 - Execute Decision Chat Completion with Prompt Execution Settings 

> "If you can’t describe what you are doing as a process, you don’t know what you’re doing." 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with 
// LLMs are trained on a diverse range of data and can provide insights on a wide range of topics like decision frameworks
// SLMS (smaller LLMs) are trained on a more specific range of data and may not provide insights on all topics
var simpleDecisionPrompt = """
Provide a list of 5 decision frameworks that can help improve the quality of decisions.
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

Here are five widely recognized decision-making frameworks that can help improve the quality of decisions:

---

### 1. **The OODA Loop (Observe, Orient, Decide, Act)**
   - **Description**: Developed by military strategist John Boyd, the OODA Loop is a cyclical process designed to help individuals and organizations make decisions quickly and adapt to changing circumstances.
   - **Steps**:
     1. **Observe**: Gather information and assess the current situation.
     2. **Orient**: Analyze the information, consider options, and align with goals.
     3. **Decide**: Choose the best course of action based on the analysis.
     4. **Act**: Implement the decision and monitor the results.
   - **Use Case**: Effective in dynamic, fast-changing environments like business strategy or crisis management.

---

### 2. **The Eisenhower Matrix (Urgent-Important Matrix)**
   - **Description**: A time-management and prioritization tool that helps focus on what truly matters by categorizing tasks bas

### Step 3 - Execute Decision Chat Completion with Streaming 

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and API services. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [4]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

Here’s a list of five proven decision-making frameworks that can help improve the quality of your decisions:

---

### 1. **OODA Loop (Observe, Orient, Decide, Act)**  
This framework, developed by military strategist John Boyd, is used to make decisions in dynamic and fast-paced environments.  

**Steps:**  
- **Observe**: Gather information about the situation.  
- **Orient**: Analyze the information and assess context, priorities, and biases.  
- **Decide**: Choose the best course of action based on the analysis.  
- **Act**: Execute the decision and observe the outcome to repeat the loop if needed.  

---

### 2. **PEST Analysis (Political, Economic, Social, Technological)**  
A strategic tool for assessing external macro-environmental factors affecting a decision or organization.  

**Steps:**  
- Consider how **Political** trends or policies might impact the decision.  
- Evaluate the **Economic** environment, such as market trends or resources.  
- Analyze **Social** aspects, in

### Step 4 - Execute Decision Chat Completion with a System Prompt & Chat History 

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt. These are usually global instructions that set the "overall rules" for interacting with the LLMs.
* User Message - A message from the user
* Assistant Message - A message from the LLM. This is a message generated from an assistant or an agent. 

Identifying the messages from which role (user) it came from can help the LLM improve its own reasoning responses. This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [5]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Simple instruction prompt to list 5 (five) decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five Decision Frameworks that can help improve the quality of decisions.
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add the assistant's response to the chat history
chatHistory.AddAssistantMessage(chatResponse);

Improving decision quality involves systematically organizing your thoughts, analyzing information, and evaluating possible outcomes. Below are five robust decision-making frameworks to help you make better decisions:

---

### 1. **Decision Matrix Analysis (Weighted Scoring Model)**  
This framework helps evaluate and prioritize multiple options based on a set of criteria.

#### **How It Works**:
1. **List Options and Criteria**: Write down all the possible options and the decision-making criteria.
2. **Assign Weights**: Allocate importance (weights) to each criterion based on its relevance.
3. **Score Options**: Rate each option against each criterion on a consistent scale (e.g., 1 to 10).
4. **Multiply and Add**: Multiply scores by weights and sum them up for each option.
5. **Select the Winner**: The option with the highest total score is your best choice.

#### **Best For**: Choosing among competing alternatives when criteria are both qualitative (e.g., "ease of use") and quantita

In the next step, an additional prompt instruction will be added to the chat history. From the 5 decision frameworks provided in the chat history, Generative AI is asked to recommed a framework best suited for the militaty intelligence community. Notice that the previous chat history is automatically provided to provide additional intelligence context. 

> "Take time to deliberate, but when the time for action comes, stop thinking and go in."
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [6]:
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestionPartTwo = """
Which of the 5 decision frameworks listed above is best suited for the military intelligence community? 
Select only the single best framework. 
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestionPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

For the military intelligence community, the **OODA Loop (Observe, Orient, Decide, Act)** is the single best-suited framework.

---

### **Reasons Why the OODA Loop Is Most Suitable**:

1. **Adaptability in Fluid Environments**:
   - Military intelligence operates in fast-paced, high-stakes environments where new information (e.g., enemy movements, emerging threats) can rapidly alter the situation.
   - The OODA Loop's iterative nature allows analysts and decision-makers to reassess and adjust in real-time, ensuring agility in dynamic scenarios.

2. **Focus on Situational Awareness**:
   - The "Observe" and "Orient" stages place a critical emphasis on gathering and interpreting intelligence—key pillars of military decision-making.
   - It helps prioritize actionable insights without getting bogged down in uncertainty.

3. **Combining Speed and Precision**:
   - In military operations, decisions must often be made rapidly, but with a calculated balance of risks and rewards. The OODA Loo

### Step 5 - Execute Decision Chat Completion with a Multi-Turn Conversation Scenario 

> "Price is what you pay. Value is what you get."  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

With Semantic Kernel maintaining state with the ChatHistory object, the natural converation between user and AI can be persisted for future use. It can be saved for future reference, it can be replayed, it can be forked in certain spots to try different scenarios, it can be used as a starting point for other conversations etc. 

In the scenario below, let's build a multi-turn conversation about a decision over a purchase for a fitness tracker. Notice that each turn the appropriate Chat History roles are populated with the appropriate prompts and assistant (AI) responses. This in effect acts as the Gathering Intelligence step of the Decision Intelligence framework as the conversation information has been persisted and can be re-used as context for the decision scenario.

In [7]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPromptDecisionConversation = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// User prompt to start a decision conversation about buying a fitness tracker
var userPromptDecisionConversationTurn1 = """
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 
""";

var assistantResponseDecisionConversationTurn1 = """
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.
""";

var userPromptDecisionConversationTurn2 = """
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?
""";

var assistantResponseDecisionConversationTurn2 = """
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.
""";

var userPromptDecisionConversationTurn3 = """
Are there any popular models you would recommend for that balance of battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn3 = """
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.
""";

var userPromptDecisionConversationTurn4 = """
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn4 = """
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 
""";

// Create a new chat history with a multi-turn conversation about buying a fitness tracker
var chatHistoryDecisionConversation = new ChatHistory();
chatHistoryDecisionConversation.AddSystemMessage(systemPromptDecisionConversation);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn1);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn1);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn2);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn2);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn3);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn3);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn4);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn4);


Using the above conversation as the "Gathered Intelligence" betweeen the user and the AI assistant, let's make a final decision and ask for final feedback and a decision evaluation. 

In [8]:
var userPromptDecisionConversationFinalDecision = """
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 
Any quick final thoughts on my decision? 
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationFinalDecision);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryDecisionConversation))
{
    chatResponse += content;
    Console.Write(content);
}

Great choice! The Fitbit Inspire 3 is a solid option for balancing affordability, battery life (up to 10 days), and reliable sleep tracking. It’s lightweight, comfortable for 24/7 wear, and integrates well with the Fitbit app for detailed insights.

**Final thoughts:**
- Make sure it’s compatible with your phone for syncing data.
- Consider any additional Fitbit Premium features (optional subscription) in case you’re interested in deeper health insights or guided programs later.
- Check for discounts or bundles since fitness trackers often go on sale!

Overall, the Inspire 3 seems aligned with your priorities. Enjoy tracking your health and sleep—it’s a great step toward better wellness! 😊

### Step 6 - Inspect Gathered Intelligence of Chat History 

The Semantic Kernel Chat History is a transparent construct that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

In [9]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistoryDecisionConversation.Count());

foreach (var message in chatHistoryDecisionConversation)
{
    Console.WriteLine(string.Empty);
    Console.WriteLine($"# {message.Role.ToString().ToUpper()}: {message.Content}");
}

Number of chat interactions: 10

# SYSTEM: You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

# USER: I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 

# ASSISTANT: Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.

# USER: Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep

Having the Chat History object available for inspection, this can be used to further help the decision-making process. For example you can load the Chat History and summarize it. 

In [10]:
var userPromptSummarizeTheDecisionChat = """
Briefly summarize the chat conversation about buying a fitness tracker.
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptSummarizeTheDecisionChat);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(userPromptSummarizeTheDecisionChat))
{
    chatResponse += content;
    Console.Write(content);
}

The chat conversation focused on choosing a fitness tracker, discussing factors like budget, features (such as heart rate monitoring, sleep tracking, and GPS), and compatibility with smartphones. Recommendations included popular brands like Fitbit, Garmin, and Apple Watch, with suggestions based on the user's specific needs and preferences, such as fitness goals and style.